## Synthetic data

In [28]:
%load_ext autoreload 
%autoreload 2

import sys
import ast

import pandas as pd
import numpy as np

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")
from src import aposteriori

import tasks.preprocessing
import tasks.run_helper

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
def parse_nullable_int_list(cell):
    try:
        parsed = ast.literal_eval(cell)
        return [int(x) if pd.notna(x) else pd.NA for x in parsed]
    except (ValueError, SyntaxError, TypeError, NameError):
        return [pd.NA]  # or [] if you prefer empty lists for invalid input


def str_list_to_list(str_list: str) -> list[str]:
    ls = str_list[1:-1].split(",")
    ls = [x.strip() if x is not None else x for x in ls]
    return ls


def age_bracket(age: int) -> str:
    if 10 <= age <= 30:
        return "Young"
    if age <= 50:
        return "Middle Aged"
    if age <= 120:
        return "Old"
    raise ValueError(f"Invalid age: {age}")


syn_df = pd.read_csv("../data/synthetic.csv")
syn_df["comment_key"] = syn_df.message + syn_df.conv_id

syn_df.Toxicity = syn_df.Toxicity.apply(str_list_to_list).apply(
    lambda x: [int(tox) for tox in x]
)

syn_df.age_annot = syn_df.age_annot.apply(parse_nullable_int_list).apply(lambda ls: [age_bracket(x) for x in ls])
syn_df.sex_annot = syn_df.sex_annot.apply(str_list_to_list)
syn_df.demographic_group_annot = syn_df.demographic_group_annot.apply(
    str_list_to_list
)
syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[Young, Young, Young, Old, Old, Old, Middle Ag...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...",@FitnessJunkie07 Respect and honesty are unive...
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","[Young, Young, Young, Old, Old, Old, Middle Ag...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I agree, but sometimes chang..."
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[Young, Young, Young, Old, Old, Old, Middle Ag...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I completely agree that resp..."
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2]","[2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","[Young, Young, Young, Old, Old, Old, Middle Ag...","['female', 'male', 'male', 'male', 'male', 'ma...","['Lesbian', 'Pansexual', 'Homosexual', 'Hetero...","['Arab', 'White', 'Caucasian', 'South Asian', ...","['Poet', 'Game Developer', 'Research Scientist...","[""Master's"", 'Some College', ""Bachelor's"", 'Ph...","@FitnessJunkie07, I understand where you're co..."
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', 'Mod

In [31]:
tasks.preprocessing.find_inconsistent_rows(syn_df)

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot,comment_key


In [32]:
discussion_df = syn_df[syn_df.conv_id == "fe858614-6571-43e7-badf-3cdcb38337b3"]
print(
    tasks.run_helper.run_aposteriori(
        discussion_df,
        feature_col="age_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    tasks.run_helper.run_aposteriori(
        discussion_df,
        feature_col="sex_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)
print(
    tasks.run_helper.run_aposteriori(
        discussion_df,
        feature_col="demographic_group_annot",
        value_col="Toxicity",
        comment_key_col="comment_key",
    )
)

Middle Aged    1.000000
Old            0.651997
Young          1.000000
dtype: float64
'female'        0.940893
'male'          1.000000
'non-binary'         NaN
dtype: float64
'Arab'                NaN
'Black'               NaN
'Caucasian'           1.0
'Indigenous'          NaN
'Latino'              NaN
'Middle Eastern'      NaN
'Pacific Islander'    NaN
'South Asian'         NaN
'White'               1.0
dtype: float64
